# Recommender systems

## One of the most common uses of big data is to predict and suggest what users may want.  This allows Google to show you relevant ads or to suggest news in Google Now; Amazon to recommend relevant products; Netflix to recommend movies that you might like; or most recently, the famous **Weekly Dicovery** of Spotify.

## All these products are based on systems of recommendation: a information retrieval method to provide users with relevant, yet novel and diverse, information. 

## In this class we will use a pretty famous dataset based on movies ratings, 'MovieLens', to learn the basics of recommender systems. 

## Table of Contents (times are approximated)

1. [Getting and analysing some data (~1:30 h)](#data)
2. [Most popular movies (~30 min)](#popular)
3. [Metrics for recommender systems (~1.30h)](#metrics)
4. [Collaborative Filtering (~15 min)](#cf)  
   4.1 [Co-occurrence Matrix (~1.30h)](#copurchase)
   <br></br>
   4.2 [Memory-based CF (~1 h)](#memory-base)
   <br></br>
   4.3 [Model-based CF (~2 h)](#model-base)

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import io
import os
import time

<a id='data'></a>
## 1.1 Load data

We will use MovieLens dataset, which is one of the most common datasets used when implementing and testing recommender engines. This data set consists of:
* 100,000 ratings (1-5) from 943 users on 1682 movies. 
* Each user has rated at least 20 movies. 
* Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens [website](https://movielens.org) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set.

You can download the dataset [here](http://files.grouplens.org/datasets/movielens/ml-100k.zip).

Take a look at the readme file!!!

In [2]:
# take a look at the readme file
data_root = "/home/dsc/repos/master-kschool/15_Sistemas_de_recomendacion/ml-100k/"
readme = os.path.join(data_root, 'README')
!cat $readme

SUMMARY & USAGE LICENSE

MovieLens data sets were collected by the GroupLens Research Project
at the University of Minnesota.
 
This data set consists of:
	* 100,000 ratings (1-5) from 943 users on 1682 movies. 
	* Each user has rated at least 20 movies. 
        * Simple demographic info for the users (age, gender, occupation, zip)

The data was collected through the MovieLens web site
(movielens.umn.edu) during the seven-month period from September 19th, 
1997 through April 22nd, 1998. This data has been cleaned up - users
who had less than 20 ratings or did not have complete demographic
information were removed from this data set. Detailed descriptions of
the data file can be found at the end of this file.

Neither the University of Minnesota nor any of the researchers
involved can guarantee the correctness of the data, its suitability
for any particular purpose, or the validity of results based on the
use of the data set.  The data set may be used for any research
purposes under th

In [34]:
# display some data
columns = ['user_id' , 'item_id' , 'rating' , 'timestamp']
datafile = os.path.join(data_root, 'u.data')
data = pd.read_csv(datafile, sep='\t', names = columns)
data.head()

,user_id,item_id,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


*Pandas library is nothing alse than numpy under the hood (numpy with steroids, if you like). You can access the data (in matrix from) with he "values" attribute, e.g. data.values*

In [35]:
# access all rows, and first 3 columns 
#[filas, columnas]
data.values[:,:3]

array([[ 196,  242,    3],
       [ 186,  302,    3],
       [  22,  377,    1],
       ...,
       [ 276, 1090,    1],
       [  13,  225,    2],
       [  12,  203,    3]])

In [36]:
# access all collumns, and first 3 rows 
data.values[:3,:]

array([[      196,       242,         3, 881250949],
       [      186,       302,         3, 891717742],
       [       22,       377,         1, 878887116]])

In [37]:
# access first 10 rows
data.values[10:, :]

array([[       62,       257,         2, 879372434],
       [      286,      1014,         5, 879781125],
       [      200,       222,         5, 876042340],
       ...,
       [      276,      1090,         1, 874795795],
       [       13,       225,         2, 882399156],
       [       12,       203,         3, 879959583]])

In [38]:
# access first column
data.values[:,0]

array([196, 186,  22, ..., 276,  13,  12])

In [39]:
# The attribute shape provides the shape of the matrix
data.values.shape

(100000, 4)

In [40]:
# Note that if we return the first column, we get a vector (of 100000 components)
data.values[:,0].shape

(100000,)

In [41]:
# same with the first row (this time, we get a vector of 4 components)
data.values[0].shape

(4,)

In [42]:
# Number of users and items
n_users = data.user_id.unique().shape[0]
n_items = data.item_id.unique().shape[0]
print('There are %s users and %s item' % (n_users, n_items))

There are 943 users and 1682 item


## 1.2 A dictionary for movies and a search tool

In order to analyze the predicted recommendations, let's create a python dictonary that will allow us to translate any item id to the corresponding movie title. Also, let's write a small function that returns the ids of the movies containing some text.

The correspondance between titles and ids is stored in the u.item file

In [43]:
# u.item file
data_root = '/home/dsc/repos/master-kschool/15_Sistemas_de_recomendacion/ml-100k/'
items_id_file = os.path.join(data_root, 'u.item')
!head $items_id_file

1|Toy Story (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Toy%20Story%20(1995)|0|0|0|1|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0
2|GoldenEye (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?GoldenEye%20(1995)|0|1|1|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
3|Four Rooms (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Four%20Rooms%20(1995)|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|0|1|0|0
4|Get Shorty (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Get%20Shorty%20(1995)|0|1|0|0|0|1|0|0|1|0|0|0|0|0|0|0|0|0|0
5|Copycat (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Copycat%20(1995)|0|0|0|0|0|0|1|0|1|0|0|0|0|0|0|0|1|0|0
6|Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)|01-Jan-1995||http://us.imdb.com/Title?Yao+a+yao+yao+dao+waipo+qiao+(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|0|0|0|0
7|Twelve Monkeys (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Twelve%20Monkeys%20(1995)|0|0|0|0|0|0|0|0|1|0|0|0|0|0|0|1|0|0|0
8|Babe (1995)|01-Jan-1995||http://us.imdb.com/M/title-exact?Babe%20(1995)|0|0|0|0|1

*Simple reminder of dictionaries*

In [27]:
# aux dictionary
aux = {'hola': 'que haces', 1:'234'}

In [28]:
# Access value of key='hola'
aux['hola']

'que haces'

In [30]:
# create new key
aux['nuevo'] = 'soy nuevo'
aux

{'hola': 'que haces', 1: '234', 'nuevo': 'soy nuevo'}

In [31]:
# Update value of existing key
aux['nuevo'] = 'ya no lo soy'
aux

{'hola': 'que haces', 1: '234', 'nuevo': 'ya no lo soy'}

In [46]:
# Create a dictionary for movie titles and ids
item_dict = {}
with io.open(items_id_file, 'rb') as f:
    for line in f.readlines():
        record = line.split(b'|')
        item_dict[int(record[0])] = str(record[1])

    
# We can use this dict to see the films a user has seen, for instance. 
for record in data.values[:20]:
    print("User {u} viewed '{m}' and gave a {r} rating".format(
        u=record[0], m=item_dict[record[1]], r=record[2]))    

User 196 viewed 'b'Kolya (1996)'' and gave a 3 rating
User 186 viewed 'b'L.A. Confidential (1997)'' and gave a 3 rating
User 22 viewed 'b'Heavyweights (1994)'' and gave a 1 rating
User 244 viewed 'b'Legends of the Fall (1994)'' and gave a 2 rating
User 166 viewed 'b'Jackie Brown (1997)'' and gave a 1 rating
User 298 viewed 'b'Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)'' and gave a 4 rating
User 115 viewed 'b'Hunt for Red October, The (1990)'' and gave a 2 rating
User 253 viewed 'b'Jungle Book, The (1994)'' and gave a 5 rating
User 305 viewed 'b'Grease (1978)'' and gave a 3 rating
User 6 viewed 'b'Remains of the Day, The (1993)'' and gave a 3 rating
User 62 viewed 'b'Men in Black (1997)'' and gave a 2 rating
User 286 viewed 'b"Romy and Michele's High School Reunion (1997)"' and gave a 5 rating
User 200 viewed 'b'Star Trek: First Contact (1996)'' and gave a 5 rating
User 210 viewed 'b'To Wong Foo, Thanks for Everything! Julie Newmar (1995)'' and gave a 3 

In [47]:
item_dict

{1: "b'Toy Story (1995)'",
 2: "b'GoldenEye (1995)'",
 3: "b'Four Rooms (1995)'",
 4: "b'Get Shorty (1995)'",
 5: "b'Copycat (1995)'",
 6: "b'Shanghai Triad (Yao a yao yao dao waipo qiao) (1995)'",
 7: "b'Twelve Monkeys (1995)'",
 8: "b'Babe (1995)'",
 9: "b'Dead Man Walking (1995)'",
 10: "b'Richard III (1995)'",
 11: "b'Seven (Se7en) (1995)'",
 12: "b'Usual Suspects, The (1995)'",
 13: "b'Mighty Aphrodite (1995)'",
 14: "b'Postino, Il (1994)'",
 15: 'b"Mr. Holland\'s Opus (1995)"',
 16: "b'French Twist (Gazon maudit) (1995)'",
 17: "b'From Dusk Till Dawn (1996)'",
 18: "b'White Balloon, The (1995)'",
 19: 'b"Antonia\'s Line (1995)"',
 20: "b'Angels and Insects (1995)'",
 21: "b'Muppet Treasure Island (1996)'",
 22: "b'Braveheart (1995)'",
 23: "b'Taxi Driver (1976)'",
 24: "b'Rumble in the Bronx (1995)'",
 25: "b'Birdcage, The (1996)'",
 26: "b'Brothers McMullen, The (1995)'",
 27: "b'Bad Boys (1995)'",
 28: "b'Apollo 13 (1995)'",
 29: "b'Batman Forever (1995)'",
 30: "b'Belle de jou

In [53]:
# Define a function that retrieves all the ids and titles for movies containing 'text' in its title
def returnItemId(text, ids):
    """
    :param text: string to be looked for in movies titles
    :param ids: dicttionary of {id:title}
    
    :return: a list of (id,title) if text found in titles, and an empty list otherwise.
    """
    # convert input text to lowercase
    text_ = text.lower()
    # find occurances
    search = [(k, v.lower().find(text_))for k, v in list(ids.items())]#me devuelve 0 o 1 segunencuentre una coincidencia o no
    # Get the IDs corresponding to the given text
    index = [k for k, v in search if v > -1]
    
    # Return a list with the id and the name
    out = []
    for i in index:
        out.append((i,ids[i]))
    return out

In [54]:
returnItemId('but', item_dict)

[(240, "b'Beavis and Butt-head Do America (1996)'"),
 (435, "b'Butch Cassidy and the Sundance Kid (1969)'"),
 (580,
  "b'Englishman Who Went Up a Hill, But Came Down a Mountain, The (1995)'"),
 (1401, "b'M. Butterfly (1993)'"),
 (1459, "b'Madame Butterfly (1995)'"),
 (1614, "b'Reluctant Debutante, The (1958)'"),
 (1621, "b'Butterfly Kiss (1995)'"),
 (1645, "b'Butcher Boy, The (1998)'"),
 (1650, "b'Butcher Boy, The (1998)'")]

## 1.3 Data consistency (always double check everything!!!)

In [56]:
# check whether titles are unique or not. They are not!!!
print(len(set(item_dict.keys())))
print(len(set(item_dict.values())))
# con esto confirmamos q hay repetidos

1682
1664


### One work around: create another dict that consolidates ids with the same movie title

In [57]:
### duplicates_item_dict ###
# Las claves en "duplicates_item_dict" son los nombres de las películas
# Los valores son una lista de los ids (que pueden ser uno solo, o varios)
duplicates_item_dict = {}

for id , name in list(item_dict.items()):
    if name not in duplicates_item_dict:
        duplicates_item_dict[name] = [id]
    else:
        duplicates_item_dict[name] = duplicates_item_dict[name] + [id]

# show the duplicated titles
for k, v in list(duplicates_item_dict.items()):
    if len(v) > 1:
        print(k,v)

b'Chasing Amy (1997)' [246, 268]
b'Kull the Conqueror (1997)' [266, 680]
b"Ulee's Gold (1997)" [297, 303]
b'Fly Away Home (1996)' [304, 500]
b'Ice Storm, The (1997)' [305, 865]
b'Deceiver (1997)' [309, 1606]
b'Desperate Measures (1998)' [329, 348]
b'Body Snatchers (1993)' [573, 670]
b'Substance of Fire, The (1996)' [711, 1658]
b'Money Talks (1997)' [876, 881]
b'That Darn Cat! (1997)' [878, 1003]
b'Hugo Pool (1997)' [1175, 1617]
b'Chairman of the Board (1998)' [1234, 1654]
b'Designated Mourner, The (1997)' [1256, 1257]
b'Hurricane Streets (1998)' [1395, 1607]
b'Sliding Doors (1998)' [1429, 1680]
b'Nightwatch (1997)' [1477, 1625]
b'Butcher Boy, The (1998)' [1645, 1650]


Create a dict where the key are the original ids, and the values are the unique one. 
We will use this dictionary to remove duplicates in a dataframe.

In [59]:
# unique_id_item_dict
unique_id_item_dict = {}
for index, lista_id in enumerate(duplicates_item_dict.values()):
    for key in lista_id:
        unique_id_item_dict[key] = index

In [60]:
unique_id_item_dict

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 33: 32,
 34: 33,
 35: 34,
 36: 35,
 37: 36,
 38: 37,
 39: 38,
 40: 39,
 41: 40,
 42: 41,
 43: 42,
 44: 43,
 45: 44,
 46: 45,
 47: 46,
 48: 47,
 49: 48,
 50: 49,
 51: 50,
 52: 51,
 53: 52,
 54: 53,
 55: 54,
 56: 55,
 57: 56,
 58: 57,
 59: 58,
 60: 59,
 61: 60,
 62: 61,
 63: 62,
 64: 63,
 65: 64,
 66: 65,
 67: 66,
 68: 67,
 69: 68,
 70: 69,
 71: 70,
 72: 71,
 73: 72,
 74: 73,
 75: 74,
 76: 75,
 77: 76,
 78: 77,
 79: 78,
 80: 79,
 81: 80,
 82: 81,
 83: 82,
 84: 83,
 85: 84,
 86: 85,
 87: 86,
 88: 87,
 89: 88,
 90: 89,
 91: 90,
 92: 91,
 93: 92,
 94: 93,
 95: 94,
 96: 95,
 97: 96,
 98: 97,
 99: 98,
 100: 99,
 101: 100,
 102: 101,
 103: 102,
 104: 103,
 105: 104,
 106: 105,
 107: 106,
 108: 107,
 109: 108,
 110: 109,
 111: 11

Create another dict mapping moving titles to this new unique id

In [61]:
# unique_item_dict
unique_item_dict = {unique_id_item_dict[k]: v for k, v in item_dict.items()}

assert(len(set(unique_item_dict.keys())) == 
       len(set(unique_item_dict.values())))

Now we can use our `returnItemId()` mehtod safely =)

In [62]:
returnItemId('but', unique_item_dict)

[(239, "b'Beavis and Butt-head Do America (1996)'"),
 (431, "b'Butch Cassidy and the Sundance Kid (1969)'"),
 (575,
  "b'Englishman Who Went Up a Hill, But Came Down a Mountain, The (1995)'"),
 (1390, "b'M. Butterfly (1993)'"),
 (1448, "b'Madame Butterfly (1995)'"),
 (1601, "b'Reluctant Debutante, The (1958)'"),
 (1607, "b'Butterfly Kiss (1995)'"),
 (1630, "b'Butcher Boy, The (1998)'")]

In [63]:
returnItemId('but', item_dict)

[(240, "b'Beavis and Butt-head Do America (1996)'"),
 (435, "b'Butch Cassidy and the Sundance Kid (1969)'"),
 (580,
  "b'Englishman Who Went Up a Hill, But Came Down a Mountain, The (1995)'"),
 (1401, "b'M. Butterfly (1993)'"),
 (1459, "b'Madame Butterfly (1995)'"),
 (1614, "b'Reluctant Debutante, The (1958)'"),
 (1621, "b'Butterfly Kiss (1995)'"),
 (1645, "b'Butcher Boy, The (1998)'"),
 (1650, "b'Butcher Boy, The (1998)'")]

## 1.4 Train and test sets

GroupLens provides several splits of the dataset, so that we can check the goodness of our algorithms. See the README file for more  details. Here we will use one of such splits.

Please notice that we have to correct for the non-unique movie's id issue!!

In [64]:
# ls data_root
!ls $data_root

allbut.pl  u1.base  u2.test  u4.base  u5.test  ub.base	u.genre  u.occupation
mku.sh	   u1.test  u3.base  u4.test  ua.base  ub.test	u.info	 u.user
README	   u2.base  u3.test  u5.base  ua.test  u.data	u.item


In [65]:
# trainfile
trainfile = os.path.join(data_root, 'ua.base')
!head $trainfile

1	1	5	874965758
1	2	3	876893171
1	3	4	878542960
1	4	3	876893119
1	5	3	889751712
1	6	5	887431973
1	7	4	875071561
1	8	1	875072484
1	9	5	878543541
1	10	3	875693118


In [87]:
# train dataframe
columns = ['user_id', 'item_id', 'rating', 'timestamp']
trainfile = os.path.join(data_root, 'ua.base')
train = pd.read_csv(trainfile, sep='\t', names=columns)
print(f'There are %s users, %s items and %s pairs in the train set' % (train.user_id.unique().shape[0], \
                                                                       train.item_id.unique().shape[0], \
                                                                       train.shape[0]))

There are 943 users, 1680 items and 90570 pairs in the train set


In [88]:
# test dataframe
columns = ['user_id', 'item_id', 'rating', 'timestamp']
testfile = os.path.join(data_root, 'ua.test')
test = pd.read_csv(testfile, sep='\t', names=columns)
print(f'There are %s users, %s items and %s pairs in the train set' % (test.user_id.unique().shape[0], \
                                                                       test.item_id.unique().shape[0], \
                                                                       test.shape[0]))

There are 943 users, 1129 items and 9430 pairs in the train set


### Correcting for non-unique movies id 

*Reminder of lambda functions in Python: is a way of calling short functions (1 line of code), without having to define the function in a separted cell.*

In [89]:
# define lambda function
def func(x):
    return unique_id_item_dict[x]

In [90]:
# Ex
test['item_id'].apply(lambda z: func(z)).head()
test['item_id'].apply(lambda z: unique_id_item_dict[z]).head()
test['item_id'].apply(func).head()
#3 formas de hacer lo mismo

0     19
1     32
2     60
3    116
4    154
Name: item_id, dtype: int64

In [91]:
# Apply lambda to train
train['item_id'] = train['item_id'].apply(lambda x: unique_id_item_dict[x])
print('Now there are %s unique items in the train set' % train.item_id.unique().shape[0])

Now there are 1662 unique items in the train set


In [92]:
# Apply lambda to test
test['item_id'] = test['item_id'].apply(func)
print('Now there are %s unique items in the test set' % test.item_id.unique().shape[0])

Now there are 1119 unique items in the test set


<a id='popular'></a>
## 2. Most popular movies

Recommending popular items is a simple, yet quite effective baseline for recommendation. Indeed, most RS suffer from a strong *popularity bias*, i.e. they tend to recommend popular items more frequently than they should -just because suggesting what is popular is effective!-. There is a lot of research  devote to understand this behaviour and to develop recipies to avoid it. 

Movies can be ranked according to different popularity metrics:
* Most rated movie (it is assumed that this is the most watched movie)
* Most positively rated movie (rating > 4.0)
* Highest rated movie

## 2.1 Most rated movie

In [93]:
# group the train dataset by item and count the number of users using Pandas
most_rated = train.groupby('item_id')['user_id'].count()

item_id
0    392
1    121
2     85
3    198
4     79
Name: user_id, dtype: int64

In [94]:
most_rated.head()

item_id
0    392
1    121
2     85
3    198
4     79
Name: user_id, dtype: int64

In [103]:
# sort in descending order
most_rated_sorted = most_rated.sort_values(ascending=False)

In [104]:
most_rated_sorted.head()

item_id
49     495
99     443
180    439
257    412
284    400
Name: user_id, dtype: int64

In [105]:
# print shape
most_rated_sorted.shape[0]

1662

In [106]:
# indices
most_rated_sorted.index

Int64Index([  49,   99,  180,  257,  284,  292,    0,  286,  120,  173,
            ...
            1525, 1551,  807, 1532, 1535, 1537, 1546, 1548, 1550, 1663],
           dtype='int64', name='item_id', length=1662)

In [110]:
# Return a numpy array of mostRatedMovies with structure [id, title, frequency]
most_rated_movies = np.zeros(shape=(most_rated_sorted.shape[0], 3), dtype=np.object)

for i, index in enumerate(most_rated_sorted.index):
    id_ = index
    freg_ = most_rated_sorted[id_]
    title_ = unique_item_dict[id_]
    most_rated_movies[i] = [id_, title_, freg_]

most_rated_movies[:10, 1:]

/tmp/ipykernel_2996/2762707652.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  most_rated_movies = np.zeros(shape=(most_rated_sorted.shape[0], 3), dtype=np.object)


array([["b'Star Wars (1977)'", 495],
       ["b'Fargo (1996)'", 443],
       ["b'Return of the Jedi (1983)'", 439],
       ["b'Contact (1997)'", 412],
       ["b'English Patient, The (1996)'", 400],
       ["b'Liar Liar (1997)'", 398],
       ["b'Toy Story (1995)'", 392],
       ["b'Scream (1996)'", 386],
       ["b'Independence Day (ID4) (1996)'", 384],
       ["b'Raiders of the Lost Ark (1981)'", 379]], dtype=object)

## 2.2 Most positively rated movie

In [113]:
# filter movies rated with rating >=4.0. Then group by item, count the number of users and sort in descending order.
positive_rated = train[train.rating >= 4.0] \
    .groupby('item_id')['user_id'].count() \
    .sort_values(ascending=False)


In [116]:
# Return a numpy array of positiveRatedMovies with structure [id, title, frequency]
most_positive_rated = np.zeros(shape=(positive_rated.shape[0], 3), dtype=np.object)

for i, index in enumerate(positive_rated.index):
    id_ = index
    freg_ = positive_rated[id_]
    title_ = unique_item_dict[id_]
    most_positive_rated[i] = [id_, title_, freg_]

most_positive_rated[:10, 1:]

/tmp/ipykernel_2996/129789981.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  most_positive_rated = np.zeros(shape=(positive_rated.shape[0], 3), dtype=np.object)


array([["b'Star Wars (1977)'", 428],
       ["b'Fargo (1996)'", 354],
       ["b'Return of the Jedi (1983)'", 331],
       ["b'Raiders of the Lost Ark (1981)'", 316],
       ["b'Silence of the Lambs, The (1991)'", 310],
       ["b'Godfather, The (1972)'", 298],
       ["b'Contact (1997)'", 276],
       ["b'Toy Story (1995)'", 275],
       ["b'Empire Strikes Back, The (1980)'", 264],
       ["b'Pulp Fiction (1994)'", 262]], dtype=object)

## 2.3 Highest mean rating movie

In [117]:
# obtaine the highest rated movies, with a minium number of users/ratings.
min_ratings = 50

# group the ratings by item and stack them in a list
list_rated_movies = train.groupby('item_id')['rating'].apply(list).reset_index()

# filter movies with a minimum number of ratings
filtered_list_rated_movies = list_rated_movies[list_rated_movies.rating.apply(lambda x: len(x) > min_ratings)]

In [118]:
filtered_list_rated_movies.head()

,item_id,rating
0,0,"[5, 4, 4, 4, 3, 1, 5, 5, 3, 5, 5, 5, 3, 5, 4, ..."
1,1,"[3, 3, 2, 3, 5, 1, 3, 3, 4, 4, 3, 2, 2, 3, 4, ..."
2,2,"[4, 2, 3, 4, 3, 2, 2, 1, 3, 3, 5, 3, 3, 3, 2, ..."
3,3,"[3, 5, 4, 5, 5, 5, 3, 5, 4, 2, 4, 4, 3, 3, 2, ..."
4,4,"[3, 1, 2, 3, 4, 4, 4, 3, 3, 2, 4, 3, 3, 4, 3, ..."


In [120]:
# obtain the mean of the list of rating per movie
mean_movies = filtered_list_rated_movies.rating.apply(lambda x: np.mean(np.array(x))).sort_values(ascending=False)

In [121]:
# Return a numpy array of meanRateMovies with structure [id, title, frequency]
mean_rate_movies = np.zeros(shape=(mean_movies.shape[0], 3), dtype=np.object)

for i, index in enumerate(mean_movies.index):
    id_ = index
    freg_ = mean_movies[id_]
    title_ = unique_item_dict[id_]
    mean_rate_movies[i] = [id_, title_, freg_]

mean_rate_movies[:10, 1:]

/tmp/ipykernel_2996/4192822962.py:2: DeprecationWarning: `np.object` is a deprecated alias for the builtin `object`. To silence this warning, use `object` by itself. Doing this will not modify any behavior and is safe. 
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  mean_rate_movies = np.zeros(shape=(mean_movies.shape[0], 3), dtype=np.object)


array([["b'Wallace & Gromit: The Best of Aardman Animation (1996)'",
        4.491525423728813],
       ["b'Close Shave, A (1995)'", 4.480769230769231],
       ["b'Wrong Trousers, The (1993)'", 4.4766355140186915],
       ['b"Schindler\'s List (1993)"', 4.4758364312267656],
       ["b'Casablanca (1942)'", 4.459821428571429],
       ["b'Shawshank Redemption, The (1994)'", 4.457364341085271],
       ["b'Usual Suspects, The (1995)'", 4.386454183266932],
       ["b'Rear Window (1954)'", 4.374358974358974],
       ["b'Star Wars (1977)'", 4.365656565656566],
       ["b'12 Angry Men (1957)'", 4.327433628318584]], dtype=object)

<div class  = "alert alert-info"> 
** QUESTION **: Which method is better?? How to measure a recommender system? 
</div>

<div class  = "alert alert-info"> 
** IMPORTANT QUESTION **: When might be useful to recommend popular items?
</div>

<a id='metrics'></a>
## 3. Metrics for recommender systems

As we have seen, even with the simplest solution --aka, recommending popular items-- is difficult to known which technique performs better. For this, there are a number of metrics that allow one to measure the goodness of a recommender system. 

Metrics can be design for measuring the relevance or accuracy of a recommendation, but they can be created for evaluating the novelty of a recommendation, or its diversity. 

For now, we will focus on relevance and accuracy. Several metrics exist:
* Accuracy: rmse, mae.
* Not ranked: Recall@k, Precision@k.
* With rank disccount: map@k, ndcg@k.
* With rank ordering: mean percentile rank.

We will be definiing some of them whitin this class. For the moment, let's talk about precision and recall.

## 3.1 Precision and recall

<img src="https://upload.wikimedia.org/wikipedia/commons/2/26/Precisionrecall.svg" alt="Precision and Recall in IR" style="float: right; width: 300px"/>

The concept of precision and recall comes form the world of information retrieval, have a look at the wikipedia:

https://en.wikipedia.org/wiki/Precision_and_recall

From this entry:

 * "**precision** (also called positive predictive value) is the fraction of retrieved instances that are relevant".
 * "**recall** (also known as sensitivity) is the fraction of relevant instances that are retrieved".

<br />
<div class  = "alert alert-info"> 
** QUESTION **: how do we know if some movie, unknown to the user, is relevant?
</div>

In other words, we cannot measure a false positive --something recommended that was not relevant--. In this regard, only recall-oriented metrics have an actual meaning in RS. Nonetheless, its common practice to define both metrics in RS as follows:
 
### $$\mathrm{recall}@N = \frac{\sum_{k=1}^N rel(k)}{\sum_{i\in \mathcal{I}_u} 1}$$
### $$\mathrm{precision}@N = \frac{\sum_{k=1}^N rel(k)}{N}$$

Here, $\mathcal{I}_u$ is the set of items adopted by user $u$, and $rel(k)$ is the relevance of a recommendation at position k in the list of recommendations. For ratings, the relevance could be defined as those movies rated above a certain threshold, e.g. $r_{ui}>4.0$. 

**Important to note: since precision is pretty much the same as recall in RS, metrcis usch as the *area under the ROC curve* doesn't have any meaning!!**

<div class = "alert alert-success">
As an example, consider a user that watched the following films:
<br /><br />
'Designated Mourner, The (1997)'
<br />
'Money Talks (1997)'
<br />
'Madame Butterfly (1995)'
<br />
'Batman Forever (1995)'
<br /><br />
The recommended items were: 
<br /><br />
'Batman (1989)' 
<br />
'Madame Butterfly (1995)'
<br /><br />
**What would be the recall and precision @1? and @2?**
<br />
**What do you think of recommending Batman? Is a bad or a good recommendation?**
</div>

Please notice that there isn't any actual difference between precision and recall in the context of RS: both measure the relevance of the recommendations, and tell nothing about items recommended that haven't been adopted by the user. Thus, it make sense to define a normalized recall as:

### $$\mathrm{recall}@N = \frac{\sum_{i=1}^N rel_i}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1})$$

This way, results are normalized to 1 always.

<div class="alert alert-success">
**Exercise** Implement the above definition of recall
</div>

In [127]:
def recall_at_n(N, test, recommended, train=None):
    """
    :param N: number of recommendations
    :param test: list of movies seen by user in test
    :param train: list of movies seen by user in train. This has to be removed from the recommended list 
    :param recommended: list of movies recommended
    
    :return the recall
    """
    if train is not None:
        rec_true = []
        for r in recommended:
            if r not in train:
                rec_true.append(r)
    else:
        rec_true = recommended
        
    intersecction = len(set(test) & set(rec_true[:N]))
    return intersecction / float(np.minimum(N, len(test)))

In [128]:
seen = ['Designated Mourner, The (1997)', 'Money Talks (1997)', 'Madame Butterfly (1995)', 'Batman Forever (1995)']
recommended = ['Batman (1989)', 'Madame Butterfly (1995)']

In [129]:
#@1
recall_at_n(1, seen, recommended)

0.0

In [130]:
#@2
recall_at_n(2, seen, recommended)

0.5

In [131]:
# Check it's well normalized
print(recall_at_n(3, seen, recommended))
print(recall_at_n(10, seen, recommended))
print(recall_at_n(100, seen, recommended))

0.3333333333333333
0.25
0.25


### Now, use this implementation to measure the efficiency of the popularity baselines in the test set. Use the top-5 movies, for instance

In [132]:
most_rated_movies[:5,1:]

array([["b'Star Wars (1977)'", 495],
       ["b'Fargo (1996)'", 443],
       ["b'Return of the Jedi (1983)'", 439],
       ["b'Contact (1997)'", 412],
       ["b'English Patient, The (1996)'", 400]], dtype=object)

In [134]:
most_positive_rated[:5,1:]

array([["b'Star Wars (1977)'", 428],
       ["b'Fargo (1996)'", 354],
       ["b'Return of the Jedi (1983)'", 331],
       ["b'Raiders of the Lost Ark (1981)'", 316],
       ["b'Silence of the Lambs, The (1991)'", 310]], dtype=object)

In [135]:
mean_rate_movies[:5,1:]

array([["b'Wallace & Gromit: The Best of Aardman Animation (1996)'",
        4.491525423728813],
       ["b'Close Shave, A (1995)'", 4.480769230769231],
       ["b'Wrong Trousers, The (1993)'", 4.4766355140186915],
       ['b"Schindler\'s List (1993)"', 4.4758364312267656],
       ["b'Casablanca (1942)'", 4.459821428571429]], dtype=object)

In [136]:
train.head()

,user_id,item_id,rating,timestamp
0,1,0,5,874965758
1,1,1,3,876893171
2,1,2,4,878542960
3,1,3,3,876893119
4,1,4,3,889751712


*Since `recall_at_n` takes both train and test list per user, we need to create a dataset with the list of movies seen in train and test*

Thus, get the list of movies per user in train and test, and join the two dataframes. For the join, use the pandas method `merge`.

In [138]:
# get movies in train per user. For this, group by user and get a list of item ids.
trainUsersGrouped = train[train.rating >= 4.0].groupby('user_id')['item_id'].apply(list).reset_index()
trainUsersGrouped

,user_id,item_id
0,1,"[0, 2, 5, 6, 8, 11, 12, 13, 14, 15, 17, 18, 21..."
1,2,"[0, 13, 24, 99, 110, 126, 236, 241, 254, 256, ..."
2,3,"[180, 259, 315, 317, 318, 324, 326, 337, 339, ..."
3,4,"[10, 257, 269, 298, 299, 321, 324, 326, 355, 3..."
4,5,"[23, 28, 39, 41, 49, 61, 69, 88, 94, 99, 100, ..."
...,...,...
937,939,"[8, 14, 117, 126, 219, 221, 236, 254, 256, 272..."
938,940,"[6, 7, 11, 49, 55, 81, 88, 94, 95, 97, 146, 16..."
939,941,"[0, 292, 296, 298, 404, 451, 910, 997]"
940,942,"[30, 49, 70, 78, 94, 96, 98, 123, 130, 171, 17..."


In [139]:
# same with test data
testUsersGrouped = test[test.rating >= 4.0].groupby('user_id')['item_id'].apply(list).reset_index()
testUsersGrouped

,user_id,item_id
0,1,"[19, 32, 60, 159, 170, 201, 264]"
1,2,"[12, 49, 250, 290, 295]"
2,3,"[325, 328]"
3,4,"[49, 259, 286, 292, 295, 350, 353, 357]"
4,5,"[0, 16]"
...,...,...
929,939,"[120, 257, 405, 472, 682, 984, 1044, 1180]"
930,940,"[65, 270, 312, 650]"
931,941,"[6, 14, 116, 123, 146, 180, 256, 257, 471, 984]"
932,942,"[116, 199, 260, 419, 423, 483, 579, 599]"


In [140]:
# make the join
joined = pd.merge(trainUsersGrouped, testUsersGrouped, how='inner', on='user_id', suffixes=('_train_', '_test_'))

In [141]:
joined.head()

,user_id,item_id_train_,item_id_test_
0,1,"[0, 2, 5, 6, 8, 11, 12, 13, 14, 15, 17, 18, 21...","[19, 32, 60, 159, 170, 201, 264]"
1,2,"[0, 13, 24, 99, 110, 126, 236, 241, 254, 256, ...","[12, 49, 250, 290, 295]"
2,3,"[180, 259, 315, 317, 318, 324, 326, 337, 339, ...","[325, 328]"
3,4,"[10, 257, 269, 298, 299, 321, 324, 326, 355, 3...","[49, 259, 286, 292, 295, 350, 353, 357]"
4,5,"[23, 28, 39, 41, 49, 61, 69, 88, 94, 99, 100, ...","[0, 16]"


In [ ]:
# Access test values:

In [142]:
# Ex, concatenate both train and test list:
joined.apply(lambda x: x[1] + x[2], axis = 1).head()

0    [0, 2, 5, 6, 8, 11, 12, 13, 14, 15, 17, 18, 21...
1    [0, 13, 24, 99, 110, 126, 236, 241, 254, 256, ...
2    [180, 259, 315, 317, 318, 324, 326, 337, 339, ...
3    [10, 257, 269, 298, 299, 321, 324, 326, 355, 3...
4    [23, 28, 39, 41, 49, 61, 69, 88, 94, 99, 100, ...
dtype: object

In [146]:
# calculate the recall of the mostRatedMovies recommendation, for each user:
joined.apply(lambda l: recall_at_n(N=15, test=l[2], recommended=most_rated_movies[:,0], train=l[1]), axis=1).head()

0    0.142857
1    0.200000
2    0.000000
3    0.375000
4    0.500000
dtype: float64

*As you can see, some users have a quite large recall (0.5), while for others is small (e.g, 0.14). Let's calculate the mean.*

In [148]:
topN = 30
# calculate the average recall across all users
recall_per_user = joined.apply(lambda l: recall_at_n(N=topN, test=l[2], recommended=most_rated_movies[:,0], train=l[1]), axis=1)

recall_per_user.mean()

0.23281797015737052

In [150]:
# recall_per_user (positiveRatedMovies case)
recall_per_user = joined.apply(lambda l: recall_at_n(N=topN, test=l[2], recommended=most_positive_rated[:,0], train=l[1]), axis=1)

recall_per_user.mean()

0.2302500764759865

In [151]:
# recall_per_user (meanRateMovies case)
recall_per_user = joined.apply(lambda l: recall_at_n(N=topN, test=l[2], recommended=mean_rate_movies[:,0], train=l[1]), axis=1)

recall_per_user.mean()

0.13298239352843227

## 3.2 Mean Averaged Precision (MAP) -- Advanced material

Previous metrics did not account for the ranking of the recommendation, i.e. the relative position of a movie within the sorted list of recommendations. **But orders matters!** Metrics like MAP, MRR or NDCG try to tackle down this problem. 

From the blog *http://fastml.com/what-you-wanted-to-know-about-mean-average-precision/*:

> Here’s another way to understand average precision. Wikipedia says AP is used to score document retrieval. You can think of it this way: you type something in Google and it shows you 10 results. It’s probably best if all of them were relevant. If only some are relevant, say five of them, then it’s much better if the relevant ones are shown first. It would be bad if first five were irrelevant and good ones only started from sixth, wouldn’t it? AP score reflects this.

Implementation taken from:

https://github.com/benhamner/Metrics/blob/master/Python/ml_metrics/average_precision.py



## Average Precision 

The Average Precision is definied as:

### $$\mathrm{AP}@N = \frac{\sum_{k=1}^N P(k) \times rel(k)}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1)}$$

where $P(k)$ is the precision at cut-off in the item list, i.e. the ratio of the number of recommended items adopted, up to the position k, over the number k. Thus:

### $$\mathrm{AP}@N = \frac{\sum_{k=1}^N \left(\sum_{i=1}^k rel(i)\right)/k \times rel(k)}{\mathrm{min}(N, \sum_{i\in \mathcal{I}_u} 1)}$$



<div class = "alert alert-success">
Following the example above, consider a user that watched the following films:
<br /><br />
'Designated Mourner, The (1997)'
<br />
'Money Talks (1997)'
<br />
'Madame Butterfly (1995)'
<br />
'Batman Forever (1995)'
<br /><br />
The recommended items were: 
<br /><br />
'Batman (1989)' 
<br />
'Madame Butterfly (1995)'
<br /><br />

<div class = "alert alert-success">
**Calculate AP@1**
<br /><br />
First, *rel(1)=0*, because Batman was not viewed. Also, *P(1) = 0*. Thus, AP@1=0.
<br />
**Calculate AP@2**
<br /><br />
As before, *rel(1)=0*, so the first term does not contribute. For the second term, *rel(2)=1*, so that *P(2)=0.5*. The numerator is hence:
<br /><br />
$P(1)*rel(1)+P(2)*rel(2)=0*0+0.5*1$
<br /><br />
For the denominator, $N=2$ and $\sum_{i\in \mathcal{I}_u} 1)=4$, thus:
<br /><br />
AP@2 = 0.5/2 = 0.25
</div>

Let's now implement it =)

In [152]:
def apk(N, test, recommended, train=None):
    """
    Computes the average precision at N given recommendations.
    
    :param N: number of recommendations
    :param test: list of movies seen by user in test
    :param train: list of movies seen by user in train. This has to be removed from the recommended list 
    :param recommended: list of movies recommended
    
    :return The average precision at N over the test set
    """
    if train is not None:
        rec_true = []
        for r in recommended:
            if r not in train:
                rec_true.append(r)
    else:
        rec_true = recommended
     
    predicted = rec_true[:N]
    score = 0.0
    num_hits = 0.0
    
    for i, p in enumerate(predicted):
        if p in test and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i + 1.0)
    
    return score / min(len(test), N)

In [153]:
seen = ['Designated Mourner, The (1997)', 'Money Talks (1997)', 'Madame Butterfly (1995)', 'Batman Forever (1995)']
recommended = ['Madame Butterfly (1995)', 'Batman (1989)']

In [154]:
# @1
apk(1, seen, recommended)

1.0

In [155]:
# @2
apk(2, seen, recommended)

0.5

In [156]:
# @3
apk(3, seen, recommended)

0.3333333333333333

## MAP

Mean avergae precision is nothing else than the AP averaged across users ;)

Apply it to popularity baselines

In [159]:
# mostRatedMovies
topN = 1
predictions = most_rated_movies[:, 0]
m1 = joined.apply(lambda l: apk(topN, l[2], predictions, l[1]), axis=1).mean()
print('map@%s=%.2f' % (topN, m1))

topN = 30
predictions = most_rated_movies[:, 0]
m1 = joined.apply(lambda l: apk(topN, l[2], predictions, l[1]), axis=1).mean()
print('map@%s=%.2f' % (topN, m1))

map@1=0.13
map@30=0.06


In [160]:
# positiveRatedMovies
topN = 1
predictions = most_positive_rated[:, 0]
m1 = joined.apply(lambda l: apk(topN, l[2], predictions, l[1]), axis=1).mean()
print('map@%s=%.2f' % (topN, m1))

topN = 30
predictions = most_positive_rated[:, 0]
m1 = joined.apply(lambda l: apk(topN, l[2], predictions, l[1]), axis=1).mean()
print('map@%s=%.2f' % (topN, m1))

map@1=0.13
map@30=0.07


In [161]:
# meanRateMovies
topN = 1
predictions = mean_rate_movies[:, 0]
m1 = joined.apply(lambda l: apk(topN, l[2], predictions, l[1]), axis=1).mean()
print('map@%s=%.2f' % (topN, m1))

topN = 30
predictions = mean_rate_movies[:, 0]
m1 = joined.apply(lambda l: apk(topN, l[2], predictions, l[1]), axis=1).mean()
print('map@%s=%.2f' % (topN, m1))

map@1=0.01
map@30=0.02


<img src="https://courses.edx.org/c4x/BerkeleyX/CS100.1x/asset/Collaborative_filtering.gif" alt="collaborative filtering" style="float: right; width: 300px"/>

## 4. Collaborative Filtering <a id='cf'></a>

Perhaps, one of the most succesful techniques for making personalized recommendations are the so called *collaborative filtering* (CF) algorithms. CF is a method of making automatic predictions (filtering) about the interests of a user by collecting preferences or taste information from many users (collaborating). The underlying assumption of the collaborative filtering approach is that if a person A has the same opinion as a person B on an issue, A is more likely to have B's opinion on a different issue X than to have the opinion on X of a person chosen randomly. 

The image at the right (from Wikipedia) shows an example of user's preference prediction using collaborative filtering. At first, people rate different items (like videos, images, games). After that, the system is making predictions about a user's rating for an item, which the user has not rated yet. These predictions are built upon the existing ratings of other users, who have similar ratings with the active user. For instance, in the image at the right the system has made a prediction, that the active user will not like the video.

In this part we will see three kinds of CF, of increasing complexity:

4.1 [CF with co-occurrence](#copurchase)

4.2 [Memory-based CF](#memory-base)

4.3 [Model-based CF](#model-base)

<a id='copurchase'></a>
## 4.1 Co-occurrence Matrix

The idea is to recommend movies similar to the movies already seen by a user. A measurement of similarity among items is obtained from the co-occurrence matrix. This is nothing else than the adjacency matrix of the graph of items created by users!!!

<table border="0" style="width:825px;border:0px;">
<tr>
    <td> 
        <img src="https://lucidworks.com/wp-content/uploads/2015/08/Les-Miserables-Co-Occurrence.png" style="width: 500px"/>
    </td>
    <td> 
        <img src="https://lucidworks.com/wp-content/uploads/2015/08/midnight-club-graph.png" style="width: 400px"/>
    </td>
</tr>
</table>


In [ ]:
# create a dictionary of movies per user
moviesPerUser = ...

In [ ]:
# calculate the number of items in train
n_items = ...

In [ ]:
# co-ocurrance matrix will have shape=[n_items,n_items]
coMatrix = ...

In [ ]:
# visualize the matrix
plt.matshow(coMatrix, fignum=1000, cmap=plt.cm.binary)
plt.gcf().set_size_inches(18.5, 10.5)
plt.show()

<div class="alert alert-success">
**QUESTION:** Can you think of a better way of visualizaing this matrix? Try to rescale it, or to rearrenge it follwoing some criteria (for instance, popularity!).
</div>

In [ ]:
popular_indexing = mostRatedMovies[:, 0].astype(np.int)
coMatrix_sorted = coMatrix[:,popular_indexing]
coMatrix_sorted_total = coMatrix_sorted[popular_indexing, :]
log_scale = np.log(coMatrix_sorted_total+1.0)
plt.matshow(log_scale, fignum=1000, cmap=plt.cm.binary)
plt.gcf().set_size_inches(18.5, 10.5)
plt.show()

### 4.1.1 Making predictions using the co-occurrence matrix

This kind of recommendations, based on item similarity, provide a measure of the closeness of one item to another. In order to make a recommendation for a user, we have to proceed as follows:

* First, define a function that returns the top-N closest items to a given one.
* Then, for a list of items adopted by a specific user, select the top-N items from the lists of top-N closest items to each adopted item.

In [ ]:
def co_occurrance_similarity(item_id, coocurrance, ntop=10):
    """
    Returns the top-N most similar items to a given one, based on the coocurrance matrix
    
    :param item_id: id of input item
    :param cooccurrance: 2-dim numpy array with the co-occurance matrix
    :param ntop: number of items to be retrieved
    
    :return top-N most similar items to the given item_id
    """
    return

In [ ]:
# First column are indices, while second one is the frequency of co-ocurrance


In [ ]:
# Play with he movie ID!!!!
queryMovieId = 180
Ntop = 5
print('For item "%s" top-%s recommendations are:' % (unique_item_dict[queryMovieId], Ntop))

similarItems = co_occurrance_similarity(queryMovieId, coMatrix, Ntop)
# let's print out the first Ntop recommendations
for r in similarItems:
    print(unique_item_dict[int(r[0])], r[1])

Now, let use this function to make recommendations:

In [ ]:
def co_occurrance_recommendation(items_id, cooccurrance, ntop=10):
    """
    Obtain the list of ntop recommendations based on a list of items (user history of views)
    
    :param items_id: list of items ids
    :param coocurrence: co-ocurrence matrix (numpy 2-dim array)
    :param ntop: top-K items to be retrieved
    
    :return list of ntop items recommended
    """
    return

In [ ]:
# get users in train with their movies
trainUsersGrouped = ...

In [ ]:
Ntop = 5
# Get the recommendations for all users using the apply method
predictions = ...

In [ ]:
positiveRatedMovies[:5]

**Note that, unlike previous popularity based models, the recommendation is now (slightly) different from one user to another**

In [ ]:
for (seen, recom) in zip(testUsersGrouped.values[:3, 1], predictions[:3]):
    print("*"*6)
    print("Seen items: ")
    print([unique_item_dict[i] for i in seen])
    print("Recommended items: ")
    print([unique_item_dict[i] for i in recom])

### Evalute the recommendation

For this, first add a column to `trainUserGrouped` with the predictions using the apply procedure above.
Next, join this dataframe with the test dataframe, and get the `recall_at_n`

In [ ]:
topN = 30
# add a prediction column to train
trainUsersGrouped['prediction'] = ...

In [ ]:
# Join the df with train and predictions with the test df
joined = ...

In [ ]:
# Uncomment the following line to see the help of method recall_at_n

# recall_at_n??

In [ ]:
# average recall across all users
recall = ...

print("recall@%s=%.3f"%(topN, recall))

In [ ]:
# average recall across all users
recall = ...

print("recall@%s=%.3f"%(topN, recall))

In [ ]:
# do the same for different top k values. It might be convenient to define a function!
def evaluate_recall(topN, trainGrouped, testGrouped, coMatrix, popularity_baseline):
    return

In [ ]:
# evaluate for N in [3,10,30,50,100]


*Do the same analysis for the map metric*

In [ ]:
# Uncomment the following line to see the help of method apk

# apk??

In [ ]:
def evaluate_map(topN, trainGrouped, testGrouped, coMatrix, popularity_baseline):
    return

In [ ]:
# evaluate for N in [3,10,30,50,100]


<div class = "alert alert-info">
Compare this results to those obtained with the popularity model. Was it so bad?
</div>

Some comments:
* The dataset we are using here is very simple. Indeed, you can see that the popularity baseline achieves quite decent metric values. This won't happen in a real world dataset! The reason it happens here is because the dataset is quite small, and quite biased towards popular items. 

* Recall does not account for the order of the recommendation, while map does. This explains why the co-occurrance model performs better after the first 10 recommendations in terms of map (ordering), while recall values are always better for the popularity based model.  


### 4.1.2 Oher distances

So far, we have defined the *closeness* of two items as the number of users shared. However, it would make make sense to define it relative the total number of users that have watch a movie. This can be done with the [Jaccard similarity index](https://en.wikipedia.org/wiki/Jaccard_index):

$$J(i,j)=\frac{|i\cap j|}{|i|+|j|-|i\cap j|}\in [0,1]$$


<div class = "alert alert-success">
Build the Jaccard similarity matrix from the co-occurrance matrix. Notice that $CoM(i,j) = |i\cap j|$ and $CoM(k,k) = |k|$. In addition, if both $|i|=0$ and $|j|=0$, the similarity is defined as 1 (this is a convention).
</div>

In [ ]:
# note that the diagonal of CoMatrix provides the number of visualizations of each movie



In [ ]:
jaccard = ...

In [ ]:
# visualize the matrix
plt.matshow(jaccard, fignum=1000, cmap=plt.cm.binary)
plt.gcf().set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
popular_indexing = mostRatedMovies[:, 0].astype(np.int)
jaccard_sorted = jaccard[:,popular_indexing]
jaccard_sorted_total = jaccard_sorted[popular_indexing, :]

# Remove ones:
jaccard_sorted_total[jaccard_sorted_total == 1.0] = 0.0
cax = plt.matshow(jaccard_sorted_total, fignum=1000, cmap=plt.cm.coolwarm)
plt.gcf().colorbar(cax, ticks=[0, 0.1, 0.2, 0.25])
plt.clim(0, 0.25)
plt.gcf().set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
queryMovieId = 40
Ntop = 5
print('For item "%s" top-%s similar items are:' % (unique_item_dict[queryMovieId], Ntop))

similarItems = co_occurrance_similarity(queryMovieId, jaccard, Ntop)
# let's print out the first Ntop recommendations
for r in similarItems:
    print(unique_item_dict[r[0]], r[1])

In [ ]:
Ntop = 10
# Calculate the predictoins with Jaccard
predictions = ...

In [ ]:
for (seen, recom) in zip(testUsersGrouped.values[:3, 1], predictions[:3]):
    print("*"*6)
    print("Seen items: ")
    print([unique_item_dict[i] for i in seen])
    print("Recommended items: ")
    print([unique_item_dict[i] for i in recom])

### Evaluate the recommendations

In [ ]:
# recall at [3,10,30,50,100]


In [ ]:
# map at [3,10,30,50,100]


**Note that the co-occurance model with Jaccard distance works better than the one counting occurances. Indeed, it has higher recall values than the popularity model for large top-N**

<div class = "alert alert-info">
** QUESTION **: Can you think of any other way of using the graph of items?
Some hints:

<br></br>
Page Rank
<br></br>
Shortest-path
<br></br>
Clustering methods: eigenvalues, spectral mehtods, etc.
</div>

<a id='memory-base'></a>
## 4.2. Memory-Based Collaborative Filtering (CF)

Although the methods developed so far return a list of recommended items, they cannot be used to make an actual prediction regarding the rating. A quite different approach would be to calculate the unknown rating, $r_{ui}$, as the averaged of some other ratings, thta are somehow close to either the user or the item in question. 

Thus, one approach is to take

### $$r_{u,i} = \frac{1}{K}\sum_{j\in\mathcal{I}'} \mathrm{sim}(i,j) r_{u,j},$$

where items $j\in\mathcal{I}'$ are taken from the set of $K$ closest items to $i$, or from the whole dataset. This is known as **item-item collaborative filtering**, and can be interpreted as *“users who liked this movie also liked …”*. See Amazon famous patent: https://www.google.com/patents/US7113917. Basically, this technique will take an item, find users who liked that item, and find other items that those users or similar users also liked. 

Similarly, one can define a **user-user filtering** where predictions are made as

### $$r_{u,i} = \frac{1}{K} \sum_{v\in\mathcal{U}'} \mathrm{sim}(u,v) r_{v,i}.$$

<img src="https://soundsuggest.files.wordpress.com/2013/06/utility_matrix.png" alt="utility matrix" style="float: right; width: 400px"/>

In this case, the recommendation would be more like *“users who are similar to you also liked …”*. Both techniques are part of the broad familiy of **Memory-Based Collaborative Filtering** approaches, or neighborhood-based algorithms.

The similarity among users or items can be calculated in a variety of forms: Pearson's correlation, cosine distance, etc. Here we will use the cosine distance. For this, we will first create the utility user-item matrix. 

The utility matrix is a dense representation of the user-item intearction. We have been using the *long* format, where missing entries are obviated; now, we will use the *wide* format, i.e. the matrix representation (see the figure on the right). 

<br></br>
<div class = "alert alert-info">
** NOTE **: Long and wide formats have its benefits and drawbacks. Can you think of some of them?
</div>

In [ ]:
train.values[:,0:3]

Put the train and test datasets in wide format (i.e., like a matrix)

In [ ]:
# utility matrix

uMatrixTraining = ...
    

In [ ]:
uMatrixTesting = ...

### Define a similarity measure: cosine similarity

### $$\mathrm{sim}({\bf a},{\bf b})=\frac{{\bf a}\cdot{\bf b}}{\sqrt{{\bf a}\cdot{\bf a}}\sqrt{{\bf  b}\cdot{\bf b}}}$$

In [ ]:
def cosineSimilarity(ratings, kind='user', epsilon=1e-9):
    """
    Calculate the cosine distance along the row (columns) of a matrix for users (items)
    
    :param ratings: a n_user X n_items matrix
    :param kind: string indicating whether we are in mode 'user' or 'item'
    :param epsilon: a small value to avoid dividing by zero (optional, defaults to 1e-9)
    
    :return a square matrix with the similarities
    """
    return

In [ ]:
# kind = 'item'


### 4.2.1. User-user CF

*“Users who are similar to you also liked …”*

### $$r_{u,i} = \frac{1}{K} \sum_{v\in\mathcal{U}'} \mathrm{sim}(u,v) r_{v,i}.$$
Consider user $x$:

1. Find other users whose ratings are “similar” to $x$’s ratings, i.e. calculate the similarity among users
2. Estimate missing ratings based on ratings of similar users

In [ ]:
# we use cosine similarity
userSimilarity = ...

In [ ]:
cax = plt.matshow(userSimilarity, fignum=1000, cmap=plt.cm.coolwarm)
plt.gcf().colorbar(cax, ticks=[0, 0.25, 0.5])
plt.clim(0, 0.5)
plt.gcf().set_size_inches(18.5, 10.5)
plt.show()

In [ ]:
np.mean(userSimilarity), np.std(userSimilarity)

*Note that if we multiply `userSimilarity` by `uMatrixTraining` we get the ratings weigthed with user similar similarity. Then, we have to normalize by the average similarity for each user*

In [ ]:
print(userSimilarity.shape, uMatrixTraining.shape)

In [ ]:
norm = ...

In [ ]:
userItemCFpredictions = ...

In [ ]:
# Be careful: take a look at the values
np.max(userItemCFpredictions), np.min(userItemCFpredictions), np.mean(userItemCFpredictions), np.std(userItemCFpredictions)

*Note that some users might give generally lower ratings than others, so that we could have also corrected for this effect as follows*

In [ ]:
sum_ = uMatrixTraining.sum(axis=1)
len_ =np.count_nonzero(uMatrixTraining, axis=1)
average_ratings = np.tile(sum_/ len_, n_items).reshape([n_items, n_users]).T

In [ ]:
uMatrixTraining_shifted = uMatrixTraining - np.multiply(average_ratings, uMatrixTraining)

In [ ]:
userItemCFpredictions_corrected = average_ratings + userSimilarity.dot(uMatrixTraining_shifted) / norm

In [ ]:
# Now rating values are more reasonable
np.max(userItemCFpredictions_corrected), np.min(userItemCFpredictions_corrected), np.mean(userItemCFpredictions_corrected), np.std(userItemCFpredictions_corrected)

### 4.2.2. Item-Item CF

*“Users who liked this movie also liked …”*

Consider item $i$:

1. For item $i$, find other similar items, i.e. calculate the similarity among items
2. Estimate rating for item $i$ based on ratings for similar items



In [ ]:
# we use cosine similarity
itemSimilarity = ...
print(itemSimilarity.shape)

In [ ]:
itemItemCFpredictions = ...

In [ ]:
itemItemCFpredictions.shape

In [ ]:
np.max(itemItemCFpredictions)

<div class="alert alert-danger">
**QUESTION:** Is averaging across all users or items computationally efficent? 
<br></br>
<br></br>
This is why nearest-neighbourghs methods (**KNN**) exists
</div>

### 4.2.3 Show some recommendations

In case of item-item CF, the recommendation is pretty much the same as with the co-occurence matrix. It's also quite simple to find similar items to a given one.

<div class="alert alert-success">
Find movies similar to a given one using the item-item similarity matrix.
</div>

In [ ]:
# let's print out the most similar items to a given item
queryMovieId = ...
print("Selected item is '%s'" % unique_item_dict[queryMovieId])


queryAnswer = ...

<div class="alert alert-success">
Calculate the recommendations obtained with the item-item CF model.
</div>

In [ ]:
# Remove relevant items seen in train from our prediction:


In [ ]:
for u in np.random.randint(0, n_users, 3):
    print("*"*6)
    print("User %s" % u)
    print("Seen items: ")
    seen = uMatrixTesting[u,:]
    print([unique_item_dict[i] for i,r in enumerate(seen) if r>4.0])
    print("Recommended items: ")
    recom = itemItemCFpredictions[u,:]
    recom = np.argsort(recom)[::-1][:10]
    print([unique_item_dict[i] for i in recom])

<div class="alert alert-success">
Do the same with the user-user CF model.
</div>

In [ ]:
for u in np.random.randint(0, n_users, 3):
    print("*"*6)
    print("User %s" % u)
    print("Seen items: ")
    seen = uMatrixTesting[u,:]
    print([unique_item_dict[i] for i,r in enumerate(seen) if r>4.0])
    print("Recommended items: ")
    recom = userItemCFpredictions[u,:]
    recom = np.argsort(recom)[::-1][:10]
    print([unique_item_dict[i] for i in recom])
    print("Recommended items (shifted version): ")
    recom = userItemCFpredictions_corrected[u,:]
    recom = np.argsort(recom)[::-1][:10]
    print([unique_item_dict[i] for i in recom])

**As you can see, the recommendations are pretty bad... Let's measure that**

### 4.2.4 Measure the recommendations

Since we are predicting ratings, it might make sense to introduce a metric that accounts for this. In particular, the **Root Mean Square Error (RMSE)** is typically used for this purpose. 

### $$\mathrm{RMSE}=\sqrt{\frac{1}{n_{\mathrm{users}}n_{\mathrm{items}}}\sum_{u,i}\left(r_{u,i}-\hat{r}_{u,i}\right)^2}$$

In [ ]:
from math import sqrt
def rmse(prediction, ground_truth):
    """
    Return the Root Mean Squared Error of the prediction
    
    :param prediction: a 2-dim numpy array with the predictions
    :param ground_truth: a 2-dim numpy array with the known ratings
    
    :return the RMSE
    """
    return

In [ ]:
print('User-based CF RMSE=%.3f' %rmse(userItemCFpredictions, uMatrixTesting))

In [ ]:
print('User-based (shifted) CF RMSE=%.3f' %rmse(userItemCFpredictions_corrected, uMatrixTesting))

In [ ]:
print('Item-based CF RMSE=%.3f' %rmse(itemItemCFpredictions, uMatrixTesting))

<div class = "alert alert-danger">
**IMPORTANT TO NOTE**: RMSE was used in the RecSys community for many years to measure the accuracy 
of recommendations. However, it was demonstrated that high accuracy in predicting rating does not imply a good
ranked list!!    
</div>

### Calculate ranking metrics

In [ ]:
userItemCFpredictions_sorted = ...

In [ ]:
# put together the rows of the test, train and predicted item ids. This can be done easily with the zip command.
# Be careful, though, because zip returns an iterable, so that it will dissapear once it's consumed. 
# To avoid this behavior, convert it to a list.



In [ ]:
# Take a look at the zipped list


<div class="alert alert-success">
As explained in class, the curse of dimensionality impedes us to obtain a good similarity measure among users or items (in this excercise, dimensions are ~1000; in a real Recommender Systems, they could be millions, billions or evene trillions!). This is the reason the above implementations do not work.
</div>

<a id='model-base'></a>
## 4.3. Model-based CF or Latent factor models
There are several model-based CF: from matrix factorizations to bayesian models, neural netwroks, etc. In all of them, we try to extract latent factors (vectors) that model user and item interactions. In contrast to previous methods, our hypothesis here is that the dimension of the latent spaces is rather small (in the order of a hundred dimensions). Then, we use this latent features to make a prediction:

## $$r_{u,i} \approx {\bf f}_u^T\cdot{\bf f}_i$$

The underlying assumption is that both users and items *live* in the same latent space, and that we can unravel such space. 

<img src="https://www.researchgate.net/profile/Tunca_Dogan/publication/235913413/figure/fig3/AS:299678856957952@1448460415040/Figure-3-The-distribution-of-the-points-in-the-Swiss-roll-dataset-in-3-D-space.png
" alt="swiss roll" style="float: center; width: 300px"/>


Here we will use a couple of linear Matrix Factorization (MF) models:

* Singular Value decomposition (SVD)
* Alternating Least Squares (ALS)

### 4.3.1 Singular value decomposition

The main idea is to reduce the dimensionality of the input space. This is pretty much the same as Eigen-decomposition or Principal Component Analysis (PCA)-

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/f/f5/GaussianScatterPCA.svg/220px-GaussianScatterPCA.svg.png
" alt="dimensionaly reducion" style="float: center; width: 500px"/>


In [ ]:
from scipy.sparse.linalg import svds

In [ ]:
# look at the help!!!


In [ ]:
#get SVD components from train matrix. Choose k.


In [ ]:
# take a look at the different matrices

# U should be an orthogonal matrix with the left singular vectors as columns

# Check U is orthogonal

# Same with V

# s is a vector with the singular values


### Get the recommendations

We will reconstruct the utility matrix R (i.e., the recommendation matrix) as follows:

### $$M\approx U\mathrm{diag}(s)V^T$$

In [ ]:
# Build a diagonal matrix with the eigenvalues

# make the prediction


In [ ]:
# check the dimensions are correct


### Evaluate the model

* RMSE
* Recall@30
* MAP@30

In [ ]:
print('SVD RMSE=%.3f' % rmse(svdPredictions, uMatrixTesting))

In [ ]:
# recall


In [ ]:
# Take a look at the zipped list


<div class = "alert alert-danger">
**IMPORTANT TO NOTE**: RMSE was used in the RecSys community for many years to measure the accuracy 
of recommendations. However, it was demonstrated that high accuracy in predicting rating does not imply a good
ranked list!!    
</div>

### Do the same with more dimensions

In [ ]:
for dim in [10, 50, 100]:
    print("*"*30)
    print("Using %s latent dimensions" %dim)
    # apply Singular Value Decomposition
    u, s, vt = svds(uMatrixTraining, dim)
    s_diag_matrix = np.diag(s)
    # make the prediction
    svdPredictions = np.dot(np.dot(u, s_diag_matrix), vt)
    print('SVD RMSE=%.3f' % rmse(svdPredictions, uMatrixTesting))
    # recall
    svdPredictions_sorted = np.argsort(svdPredictions)[::-1]
    predicted_ids = trainUsersGrouped.user_id.apply(lambda i: svdPredictions_sorted[i-1]).values
    zipped = list(zip(test_ids, train_ids, predicted_ids))
    for k in [10, 30, 50, 100]:
        recall = np.mean([recall_at_n(k,test, recom, train)  for (test, train, recom) in zipped])
        print("recall@%s=%.3f" %(k, recall))
        map_ = np.mean([apk(k,test, recom, train)  for (test, train, recom) in zipped])
        print("map@%s=%.3f" %(k, map_))

### Implicit vs Explicit feedback

In the above SVD matrix factorization we have tried to reconstructt the matrix of ratings. Howvever, it might be easier trying to model the matrix of preferences (i.e., wether the user likes or not a movie).

For this, we will define a “selector” matrix $I$ for the training utility matrix $R$, which will contain 0 if the rating matrix has no rating entry, and 1 if the rating matrix contains an entry. And the smae for testing data.


In [ ]:
# Index matrix for training data

# Index matrix for test data


In [ ]:
for dim in [50]:
    print("*"*30)
    print("Using %s latent dimensions" %dim)
    # apply Singular Value Decomposition
    u, s, vt = svds(I, dim)
    s_diag_matrix = np.diag(s)
    # make the prediction
    svdPredictions = np.dot(np.dot(u, s_diag_matrix), vt)
    print('SVD RMSE=%.3f' % rmse(svdPredictions, I2))
    # recall
    svdPredictions_sorted = np.argsort(svdPredictions)[::-1]
    predicted_ids = trainUsersGrouped.user_id.apply(lambda i: svdPredictions_sorted[i-1]).values
    zipped = list(zip(test_ids, train_ids, predicted_ids))
    for k in [10, 30, 50, 100]:
        recall = np.mean([recall_at_n(k,test, recom, train)  for (test, train, recom) in zipped])
        print("recall@%s=%.3f" %(k, recall))
        map_ = np.mean([apk(k,test, recom, train)  for (test, train, recom) in zipped])
        print("map@%s=%.3f" %(k, map_))

### Overfitting

As we introduce more dimensions in the model, we make it more prone to overfit. This can be observed in the decrease of error (RMSE) in train, while it increases in test.

In [ ]:
error_train = []
error_test = []
dims = [1, 2, 5, 7, 10, 20, 40, 80, 120, 160, 200]
for dim in dims:
    print("*"*30)
    print("Using %s latent dimensions" %dim)
    # apply Singular Value Decomposition
    u, s, vt = svds(I, dim)
    s_diag_matrix = np.diag(s)
    # make the prediction
    svdPredictions = np.dot(np.dot(u, s_diag_matrix), vt)
    error_train.append(rmse(svdPredictions, I))
    error_test.append(rmse(svdPredictions, I2))    

In [ ]:
plt.semilogx(dims, error_train, '--*b', label="train")
plt.semilogx(dims, error_test, '--*g', label="test")
plt.xlabel("Numer of latent dimensions")
plt.ylabel("Error")
plt.legend(loc=7)
plt.show()

**Note that after few dimensions (~10) the model starts to overfit (the error in test increases). Thus, we need to use other method to regularize the model (i.e., set restrictions/limitations to the model, so that it cannot overfit)**

### 3.2 Alternating Least Squares (ALS)

SVD can be very slow and computationally expensive. Besides, when addressing only the relatively few known entries we are highly prone to overfitting.

An scalable alternative to SVD is ALS, which can include regularization terms to prevent overfitting. We will rename our variable to make them more similar to the ALS notation

### ALS algorithm

The ALS algorithm aims to estimate two unknown matrices which, when multiplied together, yield the rating matrix. The loss function you will use is the well-known sum of squared errors. The second term is for regularisation to prevent overfitting

<img src="https://latex.codecogs.com/gif.latex?\underset{Q*&space;,&space;P*}{min}\sum_{(u,i)\epsilon&space;K&space;}(r_{ui}-P_u^TQ_i)^2&plus;\lambda(\left&space;\|&space;Q_i&space;\right&space;\|^2&space;&plus;&space;\left&space;\|&space;P_u&space;\right&space;\|^2)$&space;&space;$" title="\underset{q* , p*}{min}\sum_{(u,i)\epsilon K }(r_{ui}-q_i^Tp_u)^2+\lambda(\left \| q_i \right \|^2 + \left \| p_u \right \|^2)" />

The Alternating Least Squares algorithm does this by first randomly filling the users matrix with values and then optimizing the value of the movies such that the error is minimized.  Then, it holds the movies matrix constant and optimizes the value of the user's matrix.  This alternation between which matrix to optimize is the reason for the "alternating" in the name. 

<img alt="factorization" src="http://spark-mooc.github.io/web-assets/images/matrix_factorization.png" style="width: 885px"/>
<br clear="all"/>

This optimization is what's being shown on the right in the image above.  Given a fixed set of user factors (i.e., values in the users matrix), we use the known ratings to find the best values for the movie factors using the optimization written at the bottom of the figure.  Then we "alternate" and pick the best user factors given fixed movie factors.

It must be noticed that this is another way of reducing the dimensionality of the input matrix (like PCA, or more generally, SVD). This has important consequences:

* ### Our decomposition is linear. We won't be able to catch non-linear relationships among users and items.
* ### As in PCA or SVD, our features will correspond to directions of maximum variance in the data. Thus, the first feature will catch most of this variation, the second, a little bit more, and so on. It implies that the error in the reconstruction will not decrease dramatically when using more features!!! Keep this in mind.


In [ ]:
def alsRmse(I,R,P,Q):
    return np.sqrt(np.sum((I * (R - np.dot(P,Q.T)))**2)/len(R[R > 0]))

In [ ]:
def iteration(user, fixed_vecs, counts, num_factors, reg_param, num_solve, verbose=False):
    num_fixed = fixed_vecs.shape[0]
    YTY = fixed_vecs.T.dot(fixed_vecs)
    eye = np.eye(num_fixed)
    lambda_eye = reg_param * np.eye(num_factors)
    solve_vecs = np.zeros((num_solve, num_factors))

    t = time.time()
    for i in range(num_solve):
        if user:
            counts_i = counts[i]
        else:
            counts_i = counts[:, i].T
        CuI = np.diag(counts_i)
        pu = counts_i.copy()
        pu[np.where(pu != 0)] = 1.0
        YTCuIY = fixed_vecs.T.dot(CuI).dot(fixed_vecs)
        YTCupu = fixed_vecs.T.dot(CuI + eye).dot(pu.T)
        xu = np.linalg.solve(YTY + YTCuIY + lambda_eye, YTCupu)
        solve_vecs[i] = xu
        if verbose and i % 300 == 0:
            print('Solved %i vecs in %d seconds' % (i, time.time() - t))
            t = time.time()

    return solve_vecs

In [ ]:
# Check performance by plotting train and test errors
def check_als_performance(n_epochs, train_errors, test_errors):
    plt.plot(range(n_epochs), train_errors, marker='o', label='Training Data');
    plt.plot(range(n_epochs), test_errors, marker='v', label='Test Data');
    plt.title('ALS-WR Learning Curve')
    plt.xlabel('Number of Epochs');
    plt.ylabel('RMSE');
    plt.ylim(1, 5)
    plt.legend()
    plt.grid()
    plt.show()

In [ ]:
num_iterations = 20
num_factors = 50
lmbda = 0.1
T = uMatrixTesting.copy()
ptest = uMatrixTesting.copy()
ptest[ptest > 0] = 1
ptest[ptest == 0] = 0
p = uMatrixTraining.copy()
p[p > 0] = 1
p[p == 0] = 0
R = uMatrixTraining.copy()
alpha = train.shape[0] / (n_users * n_items)
C = alpha * R
user_vectors = np.random.normal(size=(n_users, num_factors))
item_vectors = np.random.normal(size=(n_items, num_factors))
train_errors = []
test_errors = []
for i in range(num_iterations):
    print('Solving for user vectors...')
    user_vectors = iteration(True, item_vectors, C, num_factors, lmbda, n_users)
    print('Solving for item vectors...')
    item_vectors = iteration(False, user_vectors, C, num_factors, lmbda, n_items)
    print('iteration %i finished' % (i + 1))
    
    train_rmse = alsRmse(p,uMatrixTraining, user_vectors,item_vectors)
    test_rmse = alsRmse(ptest,uMatrixTesting, user_vectors,item_vectors)
    train_errors.append(train_rmse)
    test_errors.append(test_rmse)
    print("[Epoch %d/%d] train error: %f, test error: %f" 
        %(i+1, num_iterations, train_rmse, test_rmse))
    
print("Algorithm converged")

In [ ]:
check_als_performance(num_iterations, train_errors, test_errors)

### ALS evaluation
* RMSE
* recall@[5, 10, 20, 50, 100, 200, 500]
* map@[5, 10, 20, 50, 100, 200, 500]

In [ ]:
alsPredictions = np.dot(user_vectors, item_vectors.T)
svdPredictions = np.dot(np.dot(u, s_diag_matrix), vt)
print('ALS CF RMSE: ' + str(rmse(alsPredictions, uMatrixTesting)))
print('SVD CF RMSE: ' + str(rmse(svdPredictions, uMatrixTesting)))

In [ ]:
# evalutation
alsPredictions_sorted = np.argsort(alsPredictions)[::-1]
predicted_ids = trainUsersGrouped.user_id.apply(lambda i: alsPredictions_sorted[i-1]).values
zipped = list(zip(test_ids, train_ids, predicted_ids))

In [ ]:
for k in [10, 30, 50, 100]:
    recall = np.mean([recall_at_n(k,test, recom, train)  for (test, train, recom) in zipped])
    print("recall@%s=%.3f" %(k, recall))
    map_ = np.mean([apk(k,test, recom, train)  for (test, train, recom) in zipped])
    print("map@%s=%.3f" %(k, map_))

Take a look at http://infolab.stanford.edu/~ullman/mmds/ch9.pdf